In [1]:
import cv2
import pandas as pd
import numpy as np
from PIL import Image
import re

def parse_srt(srt_file):
    lat_long_data = []
    with open(srt_file, 'r') as file:
        lines = file.readlines()
    
    lat_long_pattern = re.compile(r'\[latitude: ([\d.-]+)\] \[longitude: ([\d.-]+)\]')
    
    for line in lines:
        match = lat_long_pattern.search(line)
        if match:
            lat = float(match.group(1))
            long = float(match.group(2))
            lat_long_data.append((lat, long))
    
    return lat_long_data
def parse_excel(excel_file):
    df = pd.read_excel(excel_file)
    # total_length_road = df['TOTAL LENGTH OF ROAD IN ONE IMAGE'].values[0]
    # total_length_segment = df['TOTAL LENGTH OF SEGMENT'].values[0]
    total_length_road = 4.42
    total_length_segment = 610
    return total_length_road, total_length_segment
    
def video_to_unique_views(video_path, srt_file, excel_file, output_filename):
    total_length_road, total_length_segment = parse_excel(excel_file)
    total_views = int(total_length_segment / total_length_road)
    
    cap = cv2.VideoCapture(video_path)

    frames = []
    lat_long_data = parse_srt(srt_file)
    
    total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
    frame_interval = total_frames // total_views if total_views > 0 else 1

    frame_count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        if frame_count % frame_interval == 0:
            frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

            if len(lat_long_data) > frame_count // frame_interval:
                lat, long = lat_long_data[frame_count // frame_interval]
                cv2.putText(frame_rgb, f"Lat: {lat:.6f}, Long: {long:.6f}", 
                            (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (255, 255, 255), 2)

            frames.append(frame_rgb)

        frame_count += 1

    cap.release()

    if frames:
        # Stack frames vertically
        stacked_image = np.vstack(frames)

        # Convert the stacked NumPy array back to a PIL Image
        pil_image = Image.fromarray(stacked_image)

        # Save as a single TIFF file
        pil_image.save(output_filename, compression='tiff_lzw')

    print(f"Saved {len(frames)} unique views to {output_filename}.")

video_to_unique_views(
    "Data/DJI_0008.MP4", 
    "Data/DJI_0008.SRT", 
    "Data/Survey Form Segment 7.xlsx", 
    "Data/outputTiff2.jpg"
)



OSError: encoder error -2 when writing image file